In [2]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
from transformers import pipeline
from sklearn.cluster import KMeans
import folium
from sklearn.preprocessing import StandardScaler
from IPython.display import display

In [3]:
# Cargamos el DataFrame desde el archivo pickle
df = pd.read_pickle('D:/GCP/API-Proy/data/Dataset_filtrado.pickle')

In [4]:
# Filtramos los datos por estado y tipo de comida
# En el deploy cambiará por el Estado y la selección de tipo de comida del usuario
estado_seleccionado = 'Tennessee'
tipo_comida_seleccionado = 'Francés'

df_filtrado = df[(df['state'] == estado_seleccionado) & (df['category_name'] == tipo_comida_seleccionado)]

In [5]:
# Seleccionamos las columnas necesarias, excluyendo latitude y longitude de la normalización
columnas_para_escalar = [
    'business_stars', 'business_platform', 'review_stars', 'review_platform', 
    'sentimiento', 'densidad_restaurantes', 'distancia_restaurantes_populares'
]

# Escalado de características numéricas (sin incluir latitude y longitude)
scaler = StandardScaler()
df_filtrado[columnas_para_escalar] = scaler.fit_transform(df_filtrado[columnas_para_escalar])

C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_15068\930539195.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[columnas_para_escalar] = scaler.fit_transform(df_filtrado[columnas_para_escalar])


#### Modelo

In [6]:
# Aplicamos KMeans clustering para encontrar clusters de ubicaciones positivas, sin escalar latitud y longitud
locations = df_filtrado[['latitude', 'longitude']]
otras_caracteristicas = df_filtrado[columnas_para_escalar]

# Concatenamos las características sin escalar latitud y longitud
features = pd.concat([locations.reset_index(drop=True), otras_caracteristicas.reset_index(drop=True)], axis=1)

# Parámetro para personalizar el número de centroides
num_centroides = 5  # Este valor se puede cambiar según las necesidades

kmeans = KMeans(n_clusters=num_centroides, random_state=42)
kmeans.fit(features)

# Obtenemos los centroides de los clusters como las ubicaciones recomendadas
centroides = kmeans.cluster_centers_

In [7]:
centroides

array([[ 3.61082300e+01, -8.68166470e+01,  9.97664458e-02,
        -9.05012497e-01, -5.29782850e-02, -9.05012497e-01,
        -5.64834844e-01, -1.20461160e+00, -2.82581469e-01],
       [ 3.59540202e+01, -8.68244165e+01,  3.54972577e-01,
         1.10495712e+00,  8.37629642e-02,  1.10495712e+00,
         5.69756782e-01, -1.28699194e+00, -2.81365027e-01],
       [ 3.61537700e+01, -8.67979730e+01, -2.28215745e+00,
         1.10495712e+00, -5.78906166e-01,  1.10495712e+00,
         6.67007493e-01,  3.33868096e-01,  3.55560696e+00],
       [ 3.59679547e+01, -8.68207662e+01,  2.62170348e-01,
         1.10495712e+00, -1.37727520e-02,  1.10495712e+00,
         5.66809791e-01,  7.49368491e-01, -2.77936874e-01],
       [ 3.61082300e+01, -8.68166470e+01,  9.97664458e-02,
        -9.05012497e-01,  1.36355752e-01, -9.05012497e-01,
        -4.20759717e-01,  7.77509435e-01, -2.82581469e-01]])

In [8]:
# Diccionario para centrar el mapa en los estados seleccionados
centro_estados = {
    'California': [36.7783, -119.4179],
    'Florida': [27.9944024, -81.7602544],
    'Pennsylvania': [41.2033, -77.1945],
    'Tennessee': [35.5175, -86.5804],
    'Texas': [31.9686, -99.9018],
    'New York': [40.7128, -74.0060]
}

In [9]:
# Crear un mapa centrado en el estado seleccionado
centro_estado = centro_estados[estado_seleccionado]
mapa = folium.Map(location=centro_estado, zoom_start=7)

# Agregar solo los centroides (ubicaciones recomendadas) al mapa con latitud y longitud en el popup
for centroide in centroides:
    folium.Marker(
        location=[centroide[0], centroide[1]],
        popup=f'Ubicación Recomendada<br>Latitud: {centroide[0]}<br>Longitud: {centroide[1]}',
        icon=folium.Icon(color='red', icon='star')
    ).add_to(mapa)

# Mostrar el mapa en el notebook
display(mapa)

In [2]:
import requests
import numpy as np

def obtener_datos(estado: str, tipo_de_comida: str):
    url = f"https://api-modelo-nyo7ju5e4q-rj.a.run.app/recomendacion/{estado},{tipo_de_comida}"  # Asegúrate de que esta URL sea la correcta para tu API
    #params = {"estado": estado, "tipo_de_comida": tipo_de_comida}
    
    response = requests.get(url)
    
    if response.status_code == 200:
        datos = response.json()
        # Convertir los datos a un arreglo de NumPy
        arreglo = np.array(datos)
        return arreglo
    else:
        response.raise_for_status()

# Llamada de ejemplo
estado = "Tennessee"
tipo_de_comida = "Italiana"
resultado = obtener_datos(estado, tipo_de_comida)

print(type(resultado))  # Verifica que el tipo sea numpy.ndarray
resultado


<class 'numpy.ndarray'>


array([[ 3.61194787e+01, -8.67625246e+01, -8.77312102e-02,
         7.51309090e-01,  2.37027947e-01,  7.47472353e-01,
         5.73710017e-01,  0.00000000e+00, -1.59041764e-01],
       [ 3.60581089e+01, -8.35589354e+01,  7.84751639e-01,
        -1.32870274e+00,  3.72192939e-01, -1.31923302e+00,
        -2.17777550e-01,  0.00000000e+00,  5.86549801e-02],
       [ 3.61340090e+01, -8.67691076e+01, -1.31261402e+00,
         7.02633629e-01, -1.72093385e+00,  6.98588480e-01,
        -1.59472454e+00,  0.00000000e+00, -7.65156671e-02],
       [ 3.53140529e+01, -8.96670678e+01,  4.13576038e-01,
        -1.32870274e+00,  2.37900561e-01, -1.30441023e+00,
        -2.12303108e-01,  0.00000000e+00,  7.69360435e-01],
       [ 3.58750036e+01, -8.66705815e+01,  5.42155217e-01,
        -1.32870274e+00,  3.14559910e-01, -1.32804732e+00,
        -1.64802681e-01,  0.00000000e+00,  3.06727478e-01]])